# **Tutorial 4: Quantifying Uncertainty in Projections**

**Week 2, Day 1, Future Climate: The Physical Basis**

**Content creators:** Brodie Pearson, Julius Busecke, Tom Nicholas

**Content reviewers:** Younkap Nina Duplex, Zahra Khodakaramimaghsoud, Sloane Garelick, Peter Ohue, Jenna Pearson, Derick Temfack, Peizhen Yang, Cheng Zhang, Chi Zhang, Ohad Zivan

**Content editors:** Jenna Pearson, Ohad Zivan, Chi Zhang

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS

# **Tutorial Objectives**

In the previous tutorial, we took the data from a diverse set of CMIP6 models and we showed that the projections are different from one to another due to different physics, different numerics and different discretizations. In this tutorial, we will learn about the uncertainty associated with future climate projections by exploring the variability across CMIP6 model data. We will establish a 'likely' range of projections as defined by the IPCC. 

By the end of this tutorial, you will be able to 
- apply IPCC confidence levels to climate model data
- quantify the uncertainty associated with these projections.


# **Setup**

    



In [1]:
# !pip install condacolab &> /dev/null        
# import condacolab
# condacolab.install()

# # Install all packages in one call (+ use mamba instead of conda), this must in one line or code will fail
# !mamba install xarray-datatree intake-esm gcsfs xmip aiohttp nc-time-axis cf_xarray xarrayutils &> /dev/null

In [5]:
# imports
import time
tic = time.time()

import intake
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from xmip.preprocessing import combined_preprocessing
from xarrayutils.plotting import shaded_line_plot

from datatree import DataTree
from xmip.postprocessing import _parse_metric

In [6]:
# @title Figure settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")
# %matplotlib inline


In [7]:
# @title Helper functions

# If any helper functions you want to hide for clarity (that has been seen before
# or is simple/uniformative), add here
# If helper code depends on libraries that aren't used elsewhere,
# import those libaries here, rather than in the main import cell

def global_mean(ds:xr.Dataset) -> xr.Dataset:
    """Global average, weighted by the cell area"""
    return ds.weighted(ds.areacello.fillna(0)).mean(['x', 'y'], keep_attrs=True) 

# calculate anomaly to reference period
def datatree_anomaly(dt):
    dt_out = DataTree()
    for model, subtree in dt.items():
        # for the coding exercise, ellipses will go after sel on the following line
        ref = dt[model]['historical'].ds.sel(time=slice('1950', '1980')).mean() 
        dt_out[model] = subtree - ref
    return dt_out

def plot_historical_ssp126_combined(dt):
    for model in dt.keys():
        datasets = []
        for experiment in ['historical', 'ssp126']:
            datasets.append(dt[model][experiment].ds.tos)

        da_combined = xr.concat(datasets, dim='time')

In [8]:
# @title Video 4: Quantifying Uncertainty in Projections 
#Tech team will add code to format and display the video

# **Section 1: Loading the Data**

First, lets load the datasets that we used in the previous tutorial, which spanned 5 models and the experiments spanning the historical period, in addition to the low-emissions and high-emissions future scenarios



In [9]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json") # open an intake catalog containing the Pangeo CMIP cloud data

# pick our five models and three experiments
# there are many more to test out! Try executing `col.df['source_id'].unique()` to get a list of all available models
source_ids = ['IPSL-CM6A-LR', 'GFDL-ESM4', 'ACCESS-CM2', 'MPI-ESM1-2-LR', 'TaiESM1']
experiment_ids = ['historical', 'ssp126', 'ssp585']

In [13]:
# from the full `col` object, create a subset using facet search
cat = col.search(
    source_id=source_ids,
    variable_id='tos',
    member_id='r1i1p1f1',
    table_id='Omon',
    grid_label='gn',
    experiment_id = experiment_ids,
    require_all_on = ['source_id'] #make sure that we only get models which have all of the above experiments
)

# convert the sub-catalog into a datatree object, by opening each dataset into an xarray.Dataset (without loading the data)
kwargs = dict(
    preprocess=combined_preprocessing, #apply xMIP fixes to each dataset
    xarray_open_kwargs=dict(use_cftime=True), #ensure all datasets use the same time index
    storage_options={'token':'anon'} #anonymous/public authentication to google cloud storage
)

cat.esmcat.aggregation_control.groupby_attrs = ['source_id', 'experiment_id']
dt = cat.to_datatree(**kwargs)

cat_area = col.search(
    source_id=source_ids,
    variable_id='areacello', # for the coding exercise, ellipses will go after the equals on this line
    member_id='r1i1p1f1',
    table_id='Ofx', # for the coding exercise, ellipses will go after the equals on this line
    grid_label='gn',
    experiment_id = ['historical'], # for the coding exercise, ellipses will go after the equals on this line
    require_all_on = ['source_id']
)

cat_area.esmcat.aggregation_control.groupby_attrs = ['source_id', 'experiment_id']
dt_area = cat_area.to_datatree(**kwargs)

dt_with_area = DataTree()

for model,subtree in dt.items():
    metric = dt_area[model]['historical'].ds['areacello']
    dt_with_area[model] = subtree.map_over_subtree(_parse_metric,metric)

# average every dataset in the tree globally
dt_gm = dt_with_area.map_over_subtree(global_mean)

for experiment in ['historical', 'ssp126', 'ssp585']:
    da = dt_gm['TaiESM1'][experiment].ds.tos
#     da.plot(label=experiment)
# plt.title('Global Mean SST from TaiESM1')
# plt.ylabel('Global Mean SST [$^\circ$C]')
# plt.xlabel('Year')
# plt.legend()
 
# plot_historical_ssp126_combined(dt_gm)

dt_gm_anomaly = datatree_anomaly(dt_gm)

# plot_historical_ssp126_combined(dt_gm_anomaly)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id/experiment_id'


/Users/jennapearson/anaconda3/lib/python3.10/site-packages/intake_esm/_search.py:80: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in grouped:



--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id/experiment_id'


/Users/jennapearson/anaconda3/lib/python3.10/site-packages/intake_esm/_search.py:80: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in grouped:


# **Section 2: Quantifying Uncertainty in a CMIP6 Multi-model Ensemble**

Let's create a multi-model ensemble containing data from multiple CMIP6 models, which we can use to quantify our confidence in future projected sea surface temperature change under low- and high-emissions scenarios. 

Specifically, your goal is to create a *likely* range of future projected conditions. The IPCC uncertainty language defines the *likely* range as the middle 66% of model results (i.e., ignoring the upper 17% and lower 17% of results)

### **Coding Exercise 2.1: Display Multi-model Ensemble Data with IPCC Uncertainty Bands**

Complete the following code to plot:


1.   Shading to display the likely range of temperatures for the CMIP6 historical and projected data (include both *SSP1-2.6* and *SSP5-8.5*). *da_upper* and *da_lower* are the boundaries of this shaded region
2.   The multi-model mean temperature



In [14]:
#################################################
# Students: Fill in missing code (...) and comment or remove the next line
raise NotImplementedError("Student exercise: Plot the multi-model mean projections, and their likely range under different experiments/scenarios")
#################################################

for experiment, color in zip(['historical', 'ssp126', 'ssp585'], ['C0', 'C1', 'C2']):
    datasets = []
    for model in dt_gm_anomaly.keys():
        annual_sst = dt_gm_anomaly[model][experiment].ds.tos.coarsen(time=12).mean().assign_coords(source_id=model)
        datasets.append(annual_sst.sel(time=slice(None, '2100')).load()) # the french model has a long running member for ssp126 
    da = xr.concat(datasets, dim='source_id', join='override').squeeze()
    x = da.time.data
    # calculate the lower bound of the likely range
    da_lower = da.squeeze().quantile(...)
    # calculate the upper bound of the likely range
    da_upper = da.squeeze().quantile(...) 
    plt.fill_between(x, da_lower, da_upper,  alpha=0.5, color=color)
    # calculate the multi-model mean at each time within each experiment
    da.mean(...).plot(color=color, label=experiment,)
plt.title('Global Mean SST Anomaly from five-member CMIP6 ensemble (base period: 1950 to 1980)')
plt.ylabel('Global Mean SST Anomaly [$^\circ$C]')
plt.xlabel('Year')
plt.legend()

plt.show()

NotImplementedError: Student exercise: Plot the multi-model mean projections, and their likely range under different experiments/scenarios

In [ ]:
# to_remove solution
%matplotlib inline


for experiment, color in zip(['historical', 'ssp126', 'ssp585'], ['C0', 'C1', 'C2']):
    datasets = []
    for model in dt_gm_anomaly.keys():
        annual_sst = dt_gm_anomaly[model][experiment].ds.tos.coarsen(time=12).mean().assign_coords(source_id=model)
        datasets.append(annual_sst.sel(time=slice(None, '2100')).load()) # the french model has a long running member for ssp126 
    da = xr.concat(datasets, dim='source_id', join='override').squeeze()
    x = da.time.data
    # Calculate the lower bound of the likely range
    da_lower = da.squeeze().quantile(0.17, dim='source_id') 
    # Calculate the upper bound of the likely range
    da_upper = da.squeeze().quantile(0.83, dim='source_id') 
    plt.fill_between(x, da_lower, da_upper,  alpha=0.5, color=color)
    # Calculate the multi-model mean at each time within each experiment
    da.mean('source_id').plot(color=color, label=experiment) 
plt.title('Global Mean SST Anomaly from five-member \n CMIP6 ensemble (base period: 1950 to 1980)')
plt.ylabel('Global Mean SST Anomaly [$^\circ$C]')
plt.xlabel('Year')
plt.legend()

plt.show()

In [16]:
# to_remove solution
for experiment, color in zip(['historical', 'ssp126', 'ssp585'], ['C0', 'C1', 'C2']):
    datasets = []
    for model in dt_gm_anomaly.keys():
        annual_sst = dt_gm_anomaly[model][experiment].ds.tos.coarsen(time=12).mean().assign_coords(source_id=model)
        datasets.append(annual_sst.sel(time=slice(None, '2100')).load()) # the french model has a long running member for ssp126 
    da = xr.concat(datasets, dim='source_id', join='override').squeeze()
    x = da.time.data
    # calculate the lower bound of the likely range
    da_lower = da.squeeze().quantile(0.17, dim='source_id') 
    # calculate the upper bound of the likely range
    da_upper = da.squeeze().quantile(0.83, dim='source_id') 
    plt.fill_between(x, da_lower, da_upper,  alpha=0.5, color=color)
    # calculate the multi-model mean at each time within each experiment
    da.mean('source_id').plot(color=color, label=experiment) 
plt.title('Global Mean SST Anomaly from five-member \n CMIP6 ensemble (base period: 1950 to 1980)')
plt.ylabel('Global Mean SST Anomaly [$^\circ$C]')
plt.xlabel('Year')
plt.legend()

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [17]:
da_upper

<xarray.DataArray 'tos' (time: 165)>
array([-0.04864843, -0.02376341,  0.06634411,  0.0873482 ,  0.06218095,
       -0.07008726, -0.01751883,  0.0361294 , -0.01744256,  0.03377872,
        0.01732541,  0.0208975 , -0.07385085, -0.08107706, -0.00263868,
        0.00523091,  0.00616122, -0.013979  , -0.06866992, -0.10237322,
       -0.00099108,  0.03983198,  0.07899004, -0.03190746, -0.004415  ,
       -0.00872005, -0.0105167 ,  0.02202168,  0.03587501,  0.04140896,
       -0.02999576,  0.00191805,  0.03914626, -0.03425807, -0.21888499,
       -0.2406215 , -0.25524401, -0.15756546, -0.00721673, -0.03747251,
       -0.03800056, -0.03419372, -0.04970056, -0.06010155, -0.01163026,
        0.06812582,  0.00636949,  0.02076548,  0.03802866,  0.04921067,
        0.06692763,  0.00717571, -0.03903948, -0.06683086,  0.03919635,
       -0.04718645, -0.03993453, -0.06085569, -0.07679524, -0.01672759,
        0.04299411,  0.01998723, -0.10984684, -0.10654161,  0.03110917,
        0.05095725,  0.11453837,  0.02617163,  0.02728062,  0.13166295,
        0.06743804, -0.01779978,  0.1060414 ,  0.14878037,  0.07796313,
       -0.01909667,  0.01458465,  0.14226413,  0.1515448 ,  0.076667  ,
        0.10931718,  0.13605546,  0.07224576,  0.1308756 ,  0.14788009,
        0.08911226,  0.12361063,  0.05076428,  0.0646568 ,  0.10100055,
        0.1161563 ,  0.11573804,  0.12546142,  0.13404981,  0.07303762,
        0.07996308,  0.07556906,  0.10261044,  0.06315501,  0.03958466,
        0.05832728,  0.13537548,  0.15009866,  0.07699815,  0.0550823 ,
        0.04142283,  0.07957283,  0.07817334,  0.07498808,  0.07412853,
        0.1275448 ,  0.12542049,  0.07943762, -0.04685366, -0.03302733,
       -0.05648919, -0.03784456,  0.02447158,  0.03934446,  0.00383457,
       -0.00450694, -0.00557801,  0.08211591,  0.09466568,  0.00606995,
       -0.00365196,  0.03923667,  0.0524572 ,  0.07229657,  0.08205423,
        0.14294822,  0.17383601,  0.14670137,  0.04590767,  0.08800464,
        0.1658694 ,  0.20203548,  0.15991734,  0.19320278,  0.28483378,
        0.34179444,  0.29905086,  0.08435523,  0.09978506,  0.13318443,
        0.15671086,  0.30815879,  0.28195007,  0.2634101 ,  0.30154257,
        0.34754115,  0.38156557,  0.39561125,  0.42348125,  0.4203417 ,
        0.38543286,  0.39182276,  0.4207388 ,  0.44632462,  0.50760459,
        0.56784032,  0.52473115,  0.57264678,  0.55775387,  0.53325052])
Coordinates:
  * time      (time) object 1850-07-01 18:02:30 ... 2014-07-01 17:00:00
    quantile  float64 0.83

### **Questions 2.1**

1.   What does this figure tell you about how the multi-model uncertainty compares to projected physical changes in the global mean SST? 
2.   Is this the same for both scenarios?
3.   For a 5-model ensemble like this, how do the *likely* ranges specifically relate to the 5 individual model temperatures at a given time?

In [1]:
# to_remove explanation

"""
1. Under both of these scenarios (SSP1-2.6 and SSP5-8.5), the projected warming of the climate is greater than the multi-model uncertainty.
2. The warming is greater for SSP5-8.5, but so is the multi-model uncertainty, so the relative size of physical changes and multi-model uncertainty is comparable across both scenarios.
3. The likely range includes the middle 66% of model results at a given time. For 5 models this means the ranges encompass the middle 3 models (i.e., they exclude the highest and lowest cvalued model at each time).
""";

# **Summary**
In this tutorial, we have delved into quantifying the uncertainty of future climate projections by analyzing differences across various CMIP6 models. We learned to apply IPCC's confidence levels to establish a 'likely' range of projections, which refers to the middle 66% of model results. 